In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload


import sys, os, inspect
import sys
sys.path.append('/Users/emigardiner/VICO/pjams-ionization/pjams/')

from zeusmp_snapshot_reader import read_zeusmp_snapshot
from zeusmp_snapshot_reader import ScaleFactors
from snapshot import snapshot 
from basic_snapshot import basic_snapshot, VICO_loc, FREQS
import flux as flux

import numpy as np

Anglada_loc = '/Users/emigardiner/VICO/pjams-ionization/AngladaRosero2019'

In [ ]:
# INPUTS
freqs = np.array([.01, .05, .1, .5, 1, 5.3, 23, 43, 100, 230 ]) # GHz
freqs *= 10**9 # Hz
scales = np.array([500, 1000, 2000, 4000, 8000, 16000, 25000])
nums = np.array([4, 9, 21, 39, 54, 68, 94])
years = np.array([4000, 9000, 21000, 39000, 54000, 68000, 94000])
masss = np.array([1.4, 2, 4, 8, 12, 16, 24])


file = np.load(VICO_loc+'/AngladaRosero2019/AngladaData/shock_flux_arrays.npz')
lums = file['luminosities']
file.close()

In [ ]:

# snaps_lr, _, snaps_hr = flux.prep_spec_snapshots()
# for snap in np.array([snaps_lr, snaps_hr]).flatten():
#     snap.load_variabilities()

In [ ]:
rnums = np.array([9, 21, 39, 54])
snaps_lr = np.empty_like(rnums, dtype=snapshot)
snaps_hr = np.empty_like(rnums, dtype=snapshot)
for ii, rnum in enumerate(rnums):
    snaps_lr[ii] = basic_snapshot(snap=rnum, name = ('Snap%03d_nlr' % rnum))
    snaps_hr[ii] = basic_snapshot(snap=rnum, name = ('Snap%03d_nhr' % rnum))

    snaps_lr[ii].load_fluxes()
    snaps_hr[ii].load_fluxes()

In [ ]:
print(snaps_lr[-1].ScaleFluxes[0,5])
print(snaps_hr[-1].ScaleFluxes[0,5])

In [ ]:
snaps = np.empty_like(nums, dtype=snapshot)
for ii, num in enumerate(nums):
    snaps[ii] = basic_snapshot(snap=num, name = ('Snap%03d_n' % num))
    snaps[ii].load_variabilities()

In [ ]:
luminosities = np.array([899.98,
                         1317.53,
                         1301.15,
                         12499.00,
                         44323.60,
                         65461.50,
                         84459.80])

In [ ]:
print(scales)

In [ ]:
def get_flux_arrays(snapshots, case='A'):
    
    flux_1000 = np.zeros(len(snapshots))
    flux_4000 = np.zeros(len(snapshots))
    flux_25000 = np.zeros(len(snapshots))

    avg_str = 'averages' if case == 'A' else 'averages_ratio'
    for ii, snap in enumerate(snapshots):
        flux_1000[ii] = 10**snap.Variabilities[avg_str][5,1]
        flux_4000[ii] = 10**snap.Variabilities[avg_str][5,3]
        flux_25000[ii] = 10**snap.Variabilities[avg_str][5,-1]

    return flux_1000, flux_4000, flux_25000

def get_res_flux_arrays(snapshots, case='A'):
    
    flux_1000 = np.zeros(len(snapshots))
    flux_4000 = np.zeros(len(snapshots))
    flux_25000 = np.zeros(len(snapshots))


    for ii, snap in enumerate(snapshots):
        fluxes = snap.ScaleFluxes[0,5,:] if case=='A' else snap.ScaleFluxes_ratio[5,:]
        flux_1000[ii] = fluxes[1]
        flux_4000[ii] = fluxes[3]
        flux_25000[ii] = fluxes[-1]

    return flux_1000, flux_4000, flux_25000

In [ ]:
fluxA_1000, fluxA_4000, fluxA_25000 = get_flux_arrays(snaps, case='A')
fluxB_1000, fluxB_4000, fluxB_25000 = get_flux_arrays(snaps, case='B')

In [ ]:
fluxA_1000_lr, fluxA_4000_lr, fluxA_25000_lr = get_res_flux_arrays(snaps_lr, case='A')
fluxB_1000_lr, fluxB_4000_lr, fluxB_25000_lr = get_res_flux_arrays(snaps_lr, case='B')

In [ ]:
fluxA_1000_hr, fluxA_4000_hr, fluxA_25000_hr = get_res_flux_arrays(snaps_hr, case='A')
fluxB_1000_hr, fluxB_4000_hr, fluxB_25000_hr = get_res_flux_arrays(snaps_hr, case='B')

In [ ]:
print(fluxA_1000_lr)
print(fluxA_1000)
print(fluxA_1000_hr)

In [ ]:
luminosities = np.array([899.98,
                         1317.53,
                         1301.15,
                         12499.00,
                         44323.60,
                         65461.50,
                         84459.80])

In [ ]:
np.savez(Anglada_loc+'/AngladaData/shock_flux_arrays.npz', luminosities = luminosities[:],
         flux_filled_1000   =fluxA_1000,    flux_filled_4000    =fluxA_4000,    flux_filled_32000    =fluxA_25000,
         flux_filled_1000_lr=fluxA_1000_lr, flux_filled_4000_lr =fluxA_4000_lr, flux_filled_32000_lr =fluxA_25000_lr,
         flux_filled_1000_hr=fluxA_1000_hr, flux_filled_4000_hr =fluxA_4000_hr, flux_filled_32000_hr =fluxA_25000_hr,
         flux_ratio_1000    =fluxB_1000,     flux_ratio_4000    =fluxB_4000,     flux_ratio_32000    =fluxB_25000,
         flux_ratio_1000_lr =fluxB_1000_lr,  flux_ratio_4000_lr =fluxB_4000_lr,  flux_ratio_32000_lr =fluxB_25000_lr,
         flux_ratio_1000_hr =fluxB_1000_hr,  flux_ratio_4000_hr =fluxB_4000_hr,  flux_ratio_32000_hr =fluxB_25000_hr
         )

In [ ]:
luminosities = np.array([899.98,
                         1317.53,
                         1301.15,
                         12499.00,
                         44323.60,
                         65461.50,
                         84459.80])


""" 
Years            	1000au   	4000au   	32000au
Hi-res filled
9,000 yrs hi-res 	0.162599 	0.329726 	281.248527
21,000 yrs hi-res 	0.003545 	0.024687 	107.867878
39,000 yrs hi-res 	0.169222 	0.182894 	32.942426
54,000 yrs hi-res 	19.813620 	46.348223 	620.155483

Mid-Res filled
9,000 yrs mid-res 	1.793031 	1.798037 	491.569311
21,000 yrs mid-res 	1.698362 	1.698411 	30.400045
39,000 yrs mid-res 	2.968243 	3.049831 	502.691932
54,000 yrs mid-res 	7.123924 	8.303050 	127.778781

Lo-res filled
Snap009_nlr        	5.082482 	5.588887 	18.283755
Snap021_nlr        	4.809765 	4.813022 	13.289503
Snap039_nlr        	7.572746 	7.784907 	16.576450
Snap054_nlr        	8.932021 	13.108063 	21.517012

"""

# mid-res filled
flux_filled_1000 = np.array([1.277036,
                            1.793031,
                            1.698362,
                            2.968243,
                            7.123924,
                            5.209331,
                            6.920454])

flux_filled_4000 = np.array([1.970694,
                            1.798037,
                            1.698411,
                            3.049831,
                            8.303050,
                            6.868498,
                            9.354901])

flux_filled_32000 = np.array([1016.737084,
                            491.569311,
                            30.400045,
                            502.691932,
                            127.778781,
                            52.448773,
                            11.539952])

# hi-res filled
flux_filled_1000_hr = np.array([0.162599,
                                0.003545,
                                0.169222,
                                19.81362])

flux_filled_4000_hr = np.array([0.329726,
                                0.024687,
                                0.182894,
                                46.348223])

flux_filled_32000_hr = np.array([281.248527,
                                 107.867878,
                                 32.942426,
                                 620.155483])

# lo-res filled
flux_filled_1000_lr = np.array([5.082482,
                                4.809765,
                                7.572746,
                                8.932021])

flux_filled_4000_lr = np.array([5.588887,
                                4.813022,
                                7.784907,
                                13.108063])

flux_filled_32000_lr = np.array([18.283755,
                                 13.289503,
                                 16.57645,
                                 21.517012])



"""  
Years            	1000au   	4000au   	32000au
Hi-res ratio
9,000 yrs hi-res 	0.004282 	0.005285 	0.191204
21,000 yrs hi-res 	0.000097 	0.000325 	0.129702
39,000 yrs hi-res 	0.001526 	0.001951 	0.070682
54,000 yrs hi-res 	0.103195 	0.211512 	0.709238

Mid-Res ratio
9,000 yrs mid-res 	0.022141 	0.022926 	0.201473
21,000 yrs mid-res 	0.013063 	0.013076 	0.077536
39,000 yrs mid-res 	0.026858 	0.031280 	0.290204
54,000 yrs mid-res 	0.067831 	0.132885 	0.377723

Lo-res ratio
Snap009_nlr        	0.028766 	0.050783 	0.070038
Snap021_nlr        	0.022532 	0.023125 	0.031393
Snap039_nlr        	0.037043 	0.048523 	0.074351
Snap054_nlr        	0.056095 	0.146190 	0.33863

"""

#mid-res ratio
flux_ratio_1000 = np.array([0.01315900,
                            0.02214100,
                            0.01306300,
                            0.02685800,
                            0.06783100,
                            0.06259400,
                            0.06742400])

flux_ratio_4000 = np.array([0.01358700,
                            0.02292600,
                            0.01307600,
                            0.03128000,
                            0.13288500,
                            0.16140900,
                            0.16996900])

flux_ratio_32000 = np.array([0.25842000,
                            0.20147300,
                            0.07753600,
                            0.29020400,
                            0.37772300,
                            0.49217900,
                            0.46739100])

# hi-res ratio
flux_ratio_1000_hr = np.array([0.004282,
                              0.000097,
                              0.001526,
                              0.103195])

flux_ratio_4000_hr = np.array([0.005285,
                              0.000325,
                              0.001951,
                              0.211512])

flux_ratio_32000_hr = np.array([0.191204,
                              0.129702,
                              0.070682,
                              0.709238])

# lo-res ratio
flux_ratio_1000_lr = np.array([0.028766,
                              0.022532,
                              0.037043,
                              0.056095])

flux_ratio_4000_lr = np.array([0.050783,
                              0.023125,
                              0.048523,
                              0.146190])

flux_ratio_32000_lr = np.array([0.070038,
                              0.031393,
                              0.074351,
                              0.338637])

np.savez(Anglada_loc+'/AngladaData/shock_flux_arrays_WRONG.npz', luminosities = luminosities[:],
         flux_filled_1000   =flux_filled_1000,    flux_filled_4000   =flux_filled_4000,    flux_filled_32000   =flux_filled_32000,
         flux_filled_1000_lr=flux_filled_1000_lr, flux_filled_4000_lr=flux_filled_4000_lr, flux_filled_32000_lr=flux_filled_32000_lr,
         flux_filled_1000_hr=flux_filled_1000_hr, flux_filled_4000_hr=flux_filled_4000_hr, flux_filled_32000_hr=flux_filled_32000_hr,
         flux_ratio_1000    =flux_ratio_1000,     flux_ratio_4000    =flux_ratio_4000,     flux_ratio_32000    =flux_ratio_32000,
         flux_ratio_1000_lr =flux_ratio_1000_lr,  flux_ratio_4000_lr =flux_ratio_4000_lr,  flux_ratio_32000_lr =flux_ratio_32000_lr,
         flux_ratio_1000_hr =flux_ratio_1000_hr,  flux_ratio_4000_hr =flux_ratio_4000_hr,  flux_ratio_32000_hr =flux_ratio_32000_hr
         )